In [13]:
# !pip install --no-cache-dir rouge-score bert-score

In [14]:
# !pip install rouge

In [15]:
import pandas as pd
import numpy as np
import torch
import transformers
import ast
import re
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from tokenizers.normalizers import BertNormalizer
from bert_score import score as bert_score
from rouge_score import rouge_scorer

In [16]:
from rouge import Rouge
from bert_score import score as bert_score


In [17]:
def print_gpu_utilization():
    """Print GPU memory usage statistics for all available GPUs"""
    print("\nGPU Memory Usage:")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB / {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

In [18]:
def load_vocab_mappings(file_path):
    """Load vocabulary mappings from file"""
    with open(file_path, 'r', encoding='utf-8') as f:
        mappings = f.read().strip().split('\n')
    
    return {m[0]: m[2:] for m in mappings}

In [19]:
def normalize_text(text, normalizer=None, mappings=None):
    """Normalize text using MatSciBERT normalizer with improved handling"""
    # Handle empty or None input
    if not text:
        return ""
        
    # Apply basic preprocessing
    text = text.strip()
    
    # Split into lines and normalize each line
    text = [normalizer.normalize_str(s) if normalizer else s for s in text.split('\n')]
    out = []
    for s in text:
        if mappings:
            # Apply mappings with better handling of special characters
            norm_s = ''.join(mappings.get(c, c if c.isalnum() else ' ') for c in s)
            # Remove redundant spaces
            norm_s = re.sub(r'\s+', ' ', norm_s).strip()
        else:
            norm_s = s
        out.append(norm_s)
    
    return '\n'.join(out)

In [20]:
def expand_query(query):
    """Expand the query with related terms to improve matching"""
    # Simple rule-based expansion
    # Extract key terms (simple approach)
    terms = re.findall(r'\b\w+\b', query.lower())
    
    # Filter out common stop words (simplified list)
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'is', 'are', 'was', 'were', 
                 'in', 'on', 'at', 'to', 'for', 'with', 'by', 'about', 'like', 'from'}
    
    key_terms = [term for term in terms if term not in stop_words and len(term) > 2]
    
    # If no key terms found, return original query
    if not key_terms:
        return query
        
    # Add original query plus key terms for emphasis
    expanded_query = query + " " + " ".join(key_terms)
    
    return expanded_query

In [21]:
def preprocess_embeddings(embeddings):
    """Normalize document embeddings for better cosine similarity"""
    print("Normalizing document embeddings...")
    # Remove any zero vectors
    zero_indices = np.where(np.linalg.norm(embeddings, axis=1) == 0)[0]
    if len(zero_indices) > 0:
        print(f"Warning: Found {len(zero_indices)} zero vectors in embeddings. Replacing with small random values.")
        for idx in zero_indices:
            embeddings[idx] = np.random.normal(0, 0.01, embeddings.shape[1])
    
    # L2 normalize embeddings for more accurate cosine similarity
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    return normalized_embeddings

In [22]:
def get_query_embedding(query, embedding_model, tokenizer, device, pooling='mean_cls'):
    """Generate embedding for a query text using optimized pooling strategies"""
    # Apply query expansion to improve matching
    expanded_query = expand_query(query)
    
    # Normalize query (simplified, you might want to pass a normalizer if needed)
    normalized_query = normalize_text(expanded_query)
    
    # Move computation to specified device
    with torch.cuda.device(device):
        # Tokenize with increased max length for better coverage
        tokenized_query = tokenizer(normalized_query, padding=True, 
                                   truncation=True, max_length=512, return_tensors='pt').to(device)
        
        # Get embeddings with attention mask
        with torch.no_grad():
            outputs = embedding_model(**tokenized_query)
            
            # Get last hidden state and attention mask
            last_hidden_state = outputs.last_hidden_state
            attention_mask = tokenized_query['attention_mask']
            
            # Choose pooling strategy
            if pooling == 'cls':
                # CLS token pooling (first token)
                query_embedding = last_hidden_state[:, 0].cpu().numpy()[0]
                
            elif pooling == 'mean':
                # Mean pooling with attention mask
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
                sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
                sum_mask = input_mask_expanded.sum(1)
                sum_mask = torch.clamp(sum_mask, min=1e-9)
                query_embedding = (sum_embeddings / sum_mask).cpu().numpy()[0]
                
            else:  # 'mean_cls'
                # Weighted combination of CLS and mean pooling
                cls_embedding = last_hidden_state[:, 0]
                
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
                sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
                sum_mask = input_mask_expanded.sum(1)
                sum_mask = torch.clamp(sum_mask, min=1e-9)
                mean_embedding = sum_embeddings / sum_mask
                
                # Combine both methods (0.7 for mean pooling, 0.3 for CLS token)
                query_embedding = (0.7 * mean_embedding + 0.3 * cls_embedding).cpu().numpy()[0]
    
    # Normalize the embedding vector to unit length for proper cosine similarity
    query_embedding = query_embedding / np.linalg.norm(query_embedding)
    
    return query_embedding

In [23]:
def find_similar_texts(query_embedding, embeddings, documents, 
                       top_k=5, 
                       distance_metric='hybrid', 
                       similarity_threshold=0.6):
    """Find the most similar texts to the query embedding"""
    if distance_metric == 'cosine':
        # Compute cosine similarity
        similarities = cosine_similarity([query_embedding], embeddings)[0]
        
    elif distance_metric == 'dot':
        # Compute dot product
        similarities = np.dot(embeddings, query_embedding)
        
    elif distance_metric == 'hybrid':
        # Hybrid approach: combine cosine similarity and BM25-inspired weighting
        cosine_sim = cosine_similarity([query_embedding], embeddings)[0]
        
        # BM25-inspired length normalization
        k1 = 1.5
        b = 0.75
        doc_lengths = np.array([len(str(doc).split()) for doc in documents])
        avg_doc_length = np.mean(doc_lengths)
        
        length_norm = (1 - b) + b * (doc_lengths / avg_doc_length)
        bm25_weights = (k1 + 1) / (k1 * length_norm + 1)
        
        # Apply length normalization to cosine similarity
        similarities = cosine_sim * bm25_weights
    
    else:
        raise ValueError(f"Unknown distance metric: {distance_metric}")
    
    # Apply similarity threshold to filter out less relevant documents
    qualified_indices = np.where(similarities >= similarity_threshold)[0]
    
    # If no document passes the threshold, fall back to top k
    if len(qualified_indices) == 0:
        top_indices = similarities.argsort()[-top_k:][::-1]
    else:
        # Otherwise, get the top k from qualified documents
        top_qualified_similarities = similarities[qualified_indices]
        top_qualified_indices = top_qualified_similarities.argsort()[-min(top_k, len(qualified_indices)):][::-1]
        top_indices = qualified_indices[top_qualified_indices]
    
    # Get results with reranking
    results = []
    for idx in top_indices:
        doc_text = documents[idx]
        
        # Calculate additional features for reranking
        doc_length = len(str(doc_text).split())
        length_penalty = 1.0  # No penalty by default
        
        # Apply a small penalty for very short or very long documents
        if doc_length < 50:
            length_penalty = 0.9  # Slight penalty for very short docs
        elif doc_length > 1000:
            length_penalty = 0.95  # Slight penalty for very long docs
            
        # Final score combining similarity and length consideration
        final_score = similarities[idx] * length_penalty
        
        results.append({
            "index": idx,
            "text": doc_text,
            "similarity": similarities[idx],
            "final_score": final_score
        })
    
    # Final sort by the combined score
    results.sort(key=lambda x: x["final_score"], reverse=True)
    
    return results

In [24]:
def highlight_relevant_parts(text, question):
    # Extract key terms from the question
    question_terms = set(re.findall(r'\b\w+\b', question.lower()))
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'is', 'are', 'was', 'were', 
                 'in', 'on', 'at', 'to', 'for', 'with', 'by', 'about', 'like', 'from'}
    key_terms = {term for term in question_terms if term not in stop_words and len(term) > 2}
    
    if not key_terms or not text:
        return text
        
    # Split text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    # Score each sentence based on key term occurrence
    scored_sentences = []
    for sentence in sentences:
        sentence_lower = sentence.lower()
        score = sum(1 for term in key_terms if term in sentence_lower)
        scored_sentences.append((sentence, score))
    
    # Sort sentences by score (descending)
    scored_sentences.sort(key=lambda x: x[1], reverse=True)
    
    # Take top sentences and restore original order
    top_sentences = [s[0] for s in scored_sentences if s[1] > 0]
    if not top_sentences:
        # If no sentences contain key terms, return the first few sentences
        top_sentences = sentences[:3]
    
    # Get original indices to preserve document flow
    top_indices = [sentences.index(s) for s in top_sentences]
    top_indices.sort()
    
    # Reconstruct text preserving original order
    highlighted_text = " ".join([sentences[i] for i in top_indices])
    
    # If highlighted text is too short, return more of the original
    if len(highlighted_text) < len(text) * 0.3:
        return text[:1000]  # Return first 1000 chars
        
    return highlighted_text

In [25]:
    def format_context(similar_texts, question, max_context_tokens=3000):
        context = "Here is the relevant information from the dataset:\n\n"
        
        # Estimate tokens for each document and add until we hit the budget
        current_tokens = 0
        
        for i, result in enumerate(similar_texts, 1):
            doc_text = result['text']
            similarity = result['similarity']
            
            # Highlight most relevant parts 
            highlighted_text = highlight_relevant_parts(doc_text, question)
            
            # Estimate token count (rough approximation: 4 chars ~= 1 token)
            doc_token_estimate = len(highlighted_text) // 4
            
            # If adding this document would exceed the budget, truncate or skip
            if current_tokens + doc_token_estimate > max_context_tokens:
                # If it's the first document, include a truncated version
                if i == 1:
                    # Truncate to fit within remaining budget
                    max_chars = (max_context_tokens - current_tokens) * 4
                    truncated_text = highlighted_text[:max_chars] + "..."
                    context += f"Document {i} (Similarity: {similarity:.4f}):\n{truncated_text}\n\n"
                # Otherwise, we've added enough context
                break
            
            # Add the document to context
            context += f"Document {i} (Similarity: {similarity:.4f}):\n{highlighted_text}\n\n"
            current_tokens += doc_token_estimate
        
        return context

In [26]:
def evaluate_answer(answer, retrieved_docs):
    """
    Evaluate the generated answer against retrieved documents using BERTScore and ROUGE.
    
    Args:
        answer (str): The generated answer by the LLM.
        retrieved_docs (list): List of top retrieved document texts.
    
    Returns:
        dict: Dictionary containing BERTScore (P, R, F1) and ROUGE scores.
    """
    
    if not retrieved_docs:
        return {
            "bert_score": {"precision": 0.0, "recall": 0.0, "f1": 0.0},
            "rouge_scores": {"rouge-1": {"f": 0.0}, "rouge-2": {"f": 0.0}, "rouge-l": {"f": 0.0}}
        }

    # Extract texts from retrieved docs
    reference_texts = list(set(doc["text"] for doc in retrieved_docs))
    reference_text = " ".join(reference_texts)  # Concatenate retrieved docs

    # Debugging: Print reference text to check if it changes
    print("\nFinal Reference Text for ROUGE:\n", reference_text[:500], "...\n")

    try:
        # Compute BERTScore
        P, R, F1 = bert_score([answer], [reference_texts[0]], lang="en")  # Use only the most relevant document

                # Normalize text to ensure consistency
        answer = answer.lower().strip()
        reference_text = reference_text.lower().strip()


        # Compute ROUGE scores against each retrieved doc & take the best
        rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        best_rouge_scores = {"rouge-1": 0.0, "rouge-2": 0.0, "rouge-l": 0.0}

        for ref_text in reference_texts:
            scores = rouge_scorer_instance.score(ref_text, answer)
            best_rouge_scores["rouge-1"] = max(best_rouge_scores["rouge-1"], scores['rouge1'].fmeasure)
            best_rouge_scores["rouge-2"] = max(best_rouge_scores["rouge-2"], scores['rouge2'].fmeasure)
            best_rouge_scores["rouge-l"] = max(best_rouge_scores["rouge-l"], scores['rougeL'].fmeasure)

        return {
            "bert_score": {
                "precision": float(P.item()),
                "recall": float(R.item()),
                "f1": float(F1.item())
            },
            "rouge_scores": {
                "rouge-1": {"f": best_rouge_scores['rouge-1']},
                "rouge-2": {"f": best_rouge_scores['rouge-2']},
                "rouge-l": {"f": best_rouge_scores['rouge-l']}
            }
        }

    except Exception as e:
        print(f"Evaluation error: {e}")
        return {
            "bert_score": {"precision": 0.0, "recall": 0.0, "f1": 0.0},
            "rouge_scores": {"rouge-1": {"f": 0.0}, "rouge-2": {"f": 0.0}, "rouge-l": {"f": 0.0}}
        }


In [27]:
def generate_answer(question, context, llm_pipeline, max_new_tokens=512):
    """Generate answer using the LLM with improved prompt engineering"""
    # Analyze question to determine appropriate response structure
    question_lower = question.lower()
    
    # Check for question type
    is_factoid = any(w in question_lower for w in ['who', 'what', 'when', 'where', 'which', 'how many', 'how much'])
    is_comparison = any(w in question_lower for w in ['compare', 'difference', 'similar', 'versus', 'vs'])
    is_how_to = 'how to' in question_lower or 'steps' in question_lower
    
    # Build a prompt suited to the question type
    system_instruction = """You are a helpful, accurate assistant. When answering questions:
1. Only provide information supported by the context.
2. Use simple, clear language to explain complex concepts.
3. Be concise but comprehensive.
4. If the context doesn't fully answer the question, acknowledge the limitations of the information."""

    # Add specific instruction based on question type
    if is_factoid:
        system_instruction += "\nFor this factual question, cite the most relevant parts of the context."
    elif is_comparison:
        system_instruction += "\nFor this comparison question, clearly outline the similarities and differences."
    elif is_how_to:
        system_instruction += "\nFor this procedural question, provide clear step-by-step instructions."
    
    prompt = f"""{system_instruction}

Context:
{context}

Question: {question}

Answer:"""

    # Generate answer with the LLM using improved settings
    response = llm_pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.5,  # Lower temperature for more factual responses
        top_p=0.92,
        top_k=50
    )[0]['generated_text']
    
    # Extract just the answer part (after the prompt)
    answer = response[len(prompt):].strip()
    
    return answer

In [28]:
def initialize_rag_system(
    embeddings_csv_path, 
    text_column="document", 
    embedding_column="embedding",
    llm_model_id="/kaggle/input/llama-3.1/transformers/8b/2",
    top_k=5,
    distance_metric='hybrid',
    embedding_pooling ='mean_cls',
    similarity_threshold=0.55
):
    """Initialize the RAG system components"""
    # Setup environment for memory optimization
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    
    # Check GPU count
    gpu_count = torch.cuda.device_count()
    print(f"Found {gpu_count} GPUs")
    
    # Clear GPU caches
    for i in range(gpu_count):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
    
    # Load data from CSV
    print(f"Loading embeddings from {embeddings_csv_path}...")
    df = pd.read_csv(embeddings_csv_path)
    
    # Validate columns
    if text_column not in df.columns:
        raise ValueError(f"Text column '{text_column}' not found in CSV")
    
    if embedding_column not in df.columns:
        raise ValueError(f"Embedding column '{embedding_column}' not found in CSV")
    
    # Parse embeddings
    print("Parsing embeddings from string format...")
    embeddings = []
    for emb_str in df[embedding_column]:
        try:
            # Try parsing as list literal (preferred)
            emb = np.array(ast.literal_eval(emb_str))
        except (ValueError, SyntaxError):
            # Handle improperly formatted embeddings (fallback)
            emb_str = re.sub(r'\s+', ' ', emb_str.strip('[]'))  # Remove newlines & extra spaces
            try:
                emb = np.array([float(x) for x in emb_str.split()])
            except ValueError:
                print(f"Warning: Skipping malformed embedding: {emb_str[:100]}...")  # Debugging output
                continue
        embeddings.append(emb)
    
    # Convert to numpy array
    if embeddings:
        embeddings = np.vstack(embeddings)
        print(f"Loaded {len(embeddings)} embeddings with {embeddings.shape[1]} dimensions")
    else:
        raise ValueError("No valid embeddings found. Check your CSV formatting.")
    
    
    # Preprocess embeddings
    normalized_embeddings = preprocess_embeddings(embeddings)
    
    # Load text embeddings model (on GPU 0)
    print("Loading MatSciBERT for query embeddings...")
    embedding_device = "cuda:0"
    tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
    embedding_model = AutoModel.from_pretrained('m3rg-iitd/matscibert', trust_remote_code=True).to(embedding_device)
    
    # Load vocabulary mappings (optional)
    vocab_path = '/kaggle/input/vocabmappings/vocab_mappings.txt'
    mappings = load_vocab_mappings(vocab_path) if os.path.exists(vocab_path) else {}
    
    # Create normalizer
    normalizer = BertNormalizer(lowercase=False, strip_accents=True, 
                                clean_text=True, handle_chinese_chars=True)
    
    # Configure LLM loading parameters
    print(f"Loading LLM from {llm_model_id}...")
    device_map = "balanced" if gpu_count >= 2 else "auto"
    
    # Load LLM pipeline with proper configuration
    llm_pipeline = transformers.pipeline(
        "text-generation", 
        model=llm_model_id, 
        model_kwargs={
            "torch_dtype": torch.bfloat16,
            "device_map": device_map,
            "offload_folder": "/tmp/offload"
        }
    )
    
    # Return a configuration dictionary for RAG system
    return {
        "df": df,
        "embeddings": normalized_embeddings,
        "documents": df[text_column].tolist(),
        "tokenizer": tokenizer,
        "embedding_model": embedding_model,
        "embedding_device": embedding_device,
        "normalizer": normalizer,
        "mappings": mappings,
        "llm_pipeline": llm_pipeline,
        "config": {
            "top_k": top_k,
            "distance_metric": distance_metric,
            "embedding_pooling": embedding_pooling,
            "similarity_threshold": similarity_threshold
        }
    }

In [29]:
def rag_question_answering(rag_system, question):
    """
    Perform RAG-based question answering.
    """
    print("\nProcessing query...")

    # Extract configuration
    config = rag_system['config']

    # Get query embedding
    query_embedding = get_query_embedding(
        question, 
        rag_system['embedding_model'], 
        rag_system['tokenizer'], 
        rag_system['embedding_device'], 
        pooling=config.get('embedding_pooling', 'mean_cls')
    )

    # Find similar texts
    similar_texts = find_similar_texts(
        query_embedding, 
        rag_system['embeddings'], 
        rag_system['documents'], 
        top_k=config.get('top_k', 5),
        distance_metric=config.get('distance_metric', 'hybrid'), 
        similarity_threshold=config.get('similarity_threshold', 0.6)
    )

    # Format context for LLM
    context = format_context(similar_texts, question)

    # Generate answer
    answer = generate_answer(
        question, 
        context, 
        rag_system['llm_pipeline']
    )

    # Evaluate the answer
    evaluation_results = evaluate_answer(answer, similar_texts)

    return {
        "answer": answer,
        "top_results": similar_texts,
        "evaluation": evaluation_results
    }


In [30]:
def demo_rag_system():
    """Run a demonstration of the RAG system."""
    # Initialize the RAG system
    rag_system = initialize_rag_system(
        embeddings_csv_path="/kaggle/input/embeddings/chroma_embeddings.csv",
        text_column="document",
        embedding_column="embedding"
    )

    # Interactive query loop
    print("\n" + "="*50)
    print("Optimized RAG Question Answering System")
    print("="*50)
    print("Type 'exit' to quit\n")

    while True:
        question = input("\nEnter your question: ")
        if question.lower() == 'exit':
            break

        print("\nProcessing...")
        try:
            result = rag_question_answering(rag_system, question)

            print("\n" + "="*50)
            print("ANSWER:")
            print("="*50)
            print(result["answer"])

            print("\n" + "="*50)
            print("TOP RELEVANT DOCUMENTS:")
            print("="*50)
            for i, doc in enumerate(result["top_results"], 1):
                print(f"{i}. Similarity: {doc['similarity']:.4f}, Final Score: {doc['final_score']:.4f}")
                print(f"Retrieved Text:\n{doc['text'][:500]}...\n")  # Print first 500 characters
                text_preview = doc["text"]
                if len(text_preview) > 300:
                    text_preview = text_preview[:300] + "..."
                print(f"{text_preview}\n")

            # Print Evaluation Scores
            evaluation = result["evaluation"]
            print("\n" + "="*50)
            print("EVALUATION SCORES:")
            print("="*50)
            print(f"BERTScore Precision: {evaluation['bert_score']['precision']:.4f}")
            print(f"BERTScore Recall: {evaluation['bert_score']['recall']:.4f}")
            print(f"BERTScore F1: {evaluation['bert_score']['f1']:.4f}")
            print(f"ROUGE-1: {evaluation['rouge_scores']['rouge-1']['f']:.4f}")
            print(f"ROUGE-2: {evaluation['rouge_scores']['rouge-2']['f']:.4f}")
            print(f"ROUGE-L: {evaluation['rouge_scores']['rouge-l']['f']:.4f}")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print("\nERROR: CUDA out of memory. Try clearing GPU cache and reducing model parameters.")

                # Clear GPU caches
                for i in range(torch.cuda.device_count()):
                    with torch.cuda.device(i):
                        torch.cuda.empty_cache()
                        torch.cuda.synchronize()

                print_gpu_utilization()
            else:
                print(f"\nERROR: {str(e)}")


In [31]:
demo_rag_system()

Found 2 GPUs
Loading embeddings from /kaggle/input/embeddings/chroma_embeddings.csv...
Parsing embeddings from string format...
Loaded 200 embeddings with 768 dimensions
Normalizing document embeddings...
Loading MatSciBERT for query embeddings...


tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/467k [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Processing...

Processing query...

Final Reference Text for ROUGE:
 37. The paper Simultaneously Increasing the Strength and Ductility of a Refractory
HighEntropy Alloy via Grain Refining investigates the grain growth kinetics and tensile
properties of the HfNbTaTiZr refractory high-entropy alloy. The HfNbTaTiZr
refractory high-entropy alloy was investigated on the grain growth kinetics and tensile
properties. Grain growth at 1200-1350 °C is rather slow. The activation energy is 389
kJ/mol and the growth exponent is 3.5. The HfNbTaTiZr alloy has high strength, s ...



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



ANSWER:
The yield strength of Al0.5NbTiMoV is 58 MPa.
Explanation: The yield strength of Al0.5NbTiMoV is 58 MPa. This information can be found in Document 4, which is the most relevant document for this question. Document 4 provides the yield strengths of HfMo0.5NbTiV0.5 and HfMo0.5NbTiV0.5Si0.7 alloys at 1200 °C, and the yield strength of Al0.5NbTiMoV can be calculated by taking the average of these two values. The yield strength of HfMo0.5NbTiV0.5 is 60 MPa, and the yield strength of HfMo0.5NbTiV0.5Si0.7 is 235 MPa. Therefore, the yield strength of Al0.5NbTiMoV is 58 MPa.

TOP RELEVANT DOCUMENTS:
1. Similarity: 0.8252, Final Score: 0.8252
Retrieved Text:
estimated elastic parameters are found to be rather close to the values obtained from the CPA
and SQS calculations, especially on the scale of the present values (0-250 GPa). But this simple
estimate turned out to be too rough to resolve delicate alloying effects like the one seen for C44
of TiZrNbMo and TiZrVNbMo. Nevertheless, whe


Enter your question:  exit


**Complete code**
